In [ ]:
from covidvu.cryostation import Cryostation

In [ ]:
import pandas as pd

In [ ]:
from covidvu.pipeline.vujson import parseCSSE

In [ ]:
confirmedCases = parseCSSE('confirmed')['casesGlobal']

In [ ]:
storage = Cryostation('database/virustrack.db')

countries = []
for c in [k for k in storage.keys()]:
    if c[0] != '!':
        countries.append(c)

In [ ]:
target = 'confirmed'

In [ ]:
regionTS = pd.Series(storage[countries[0]][target])
regionTS.index = pd.to_datetime(regionTS.index)

In [ ]:
minCasesFilter = 100

In [ ]:
(regionTS > minCasesFilter).argmax()

In [ ]:
storage.close()I 

In [ ]:
storage.close()

In [ ]:
statesUS = []
for s in [p for p in storage['US']['provinces'].keys()]:
    if s[0] != '!':
        statesUS.append(s)

In [ ]:
countries

In [ ]:
statesUS

In [ ]:
from covidvu.predict import buildLogisticModel

logGrowthModel = buildLogisticModel()

from covidvu.predict import predictRegions

import os

In [ ]:
TEST_SITE_DATA = os.path.join(os.getcwd(), 'resources', 'test_site_data')
TEST_N_SAMPLES = 1000
TEST_N_CHAINS = 2

In [ ]:
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
predictions = predictRegions('US',
               nDaysPredict=10,
               siteData=TEST_SITE_DATA,                       
               logGrowthModel=logGrowthModel,
               nSamples=TEST_N_SAMPLES,
               nChains=TEST_N_CHAINS,
               )

In [ ]:
predictions

In [ ]:
import json
from os.path import join

In [ ]:
def _assertValidJSON(fname):
    assert os.path.exists(fname)
    with open(fname) as f:
        jsonObject = json.load(f)
    assert isinstance(jsonObject, dict)
    assert len(jsonObject.keys()) > 0

In [ ]:
_assertValidJSON(join(TEST_SITE_DATA,'prediction-world-mean-US.json'))
_assertValidJSON(join(TEST_SITE_DATA, 'prediction-world-conf-int-US.json'))

In [ ]:
2 > None

In [ ]:
from covidvu.config import MASTER_DATABASE

In [ ]:
c = Cryostation(MASTER_DATABASE)

In [ ]:
country = c['US']

In [ ]:
'provinces' in country

In [ ]:
'info' in country

In [ ]:
'region' in country

In [ ]:
country['provinces']['California']['confirmed']['2020-04-01']

In [ ]:
c.close()

In [ ]:
storage.close()